# PMR3508 - Aprendizado de Máquina e Reconhecimento de Padrões

## EP3: Análise de Sentimentos com RNA's e Doc2Vec na base "Large Movie Review"

## Autor: Rodrigo Gebara Reis, Hash 173

Nesse EP, o objetivo é compreender o uso de modelos como Doc2Vec para criar uma Rede Neural Artificial para analisar o sentimento (positivo ou negativo) de opiniões sobre filmes. Essas opiniões foram retiradas do IMDb, um dos maiores repositórios de críticas de filmes e séries do mundo. Além disso, serão exploradas diferenças de performance entre as RNA's criadas com sklearn e TensorFlow, além de outros modelos alternativos.

# 1. Configuração:

## 1.1 Instalação de pacotes:

Inicialmente, devemos instalar dois pacotes que serão muito usados no decorrer do notebook: $\textit{ftfy}$ (Fixes Text For You), que corrige alguns erros de transcrição do Unicode; e $\textit{gensim}$, importante em trabalhos relacionados a Natural Language Processing. Finalmente, vamos instalar também $\textit{tensorflow}$ e $\textit{keras}$, responsáveis por algumas redes neurais.

In [1]:
!pip3 install ftfy
!pip3 install gensim
!pip3 install tensorflow
!pip3 install keras

     |████████████████████████████████| 64 kB 597 kB/s 
  Created wheel for ftfy: filename=ftfy-5.8-py3-none-any.whl size=45612 sha256=b0ba44d9d5e7a6e255bb6b51d3148a7c1472fbc9f66b57c84d4015559fe9b8dc
  Stored in directory: /root/.cache/pip/wheels/49/1c/fc/8b19700f939810cd8fd9495ae34934b246279791288eda1c31
Successfully built ftfy


Agora, vamos importar as bibliotecas pertinentes, que serão usadas no decorrer do notebook:

In [2]:
#Geral
import numpy as np
np.random.seed(36)
import pandas as pd
import matplotlib.pyplot as plt
import random
random.seed(36)

#Processamento de textos
from ftfy import fix_text
import string
import re
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#ML e NLP
from sklearn.linear_model import LogisticRegression
from sklearn import neural_network, svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
tf.random.set_seed(36)
import keras
from keras import Sequential, regularizers
from keras.layers import Dense
from tensorflow.python.framework import ops
from keras.callbacks import EarlyStopping
from tqdm import tqdm as tqdm

Note que foram configuradas seeds para os comandos de aleatoriedade, de modo a possibilitar a reprodução dos resultados da análise.

# 2. Data prep:

## 2.1. Importação dos dados:

Importando as bases de treino, validação e teste, respectivamente:

In [3]:
train = pd.read_csv("../input/sentiment-analysis-pmr3508/data_train.csv")

## 2.2. Compreensão e análise inicial do dataset:

Agora vamos visualizar as primeiras linhas do dataframe de treino, para conhecer seus atributos. Além disso, devemos conferir seu tamanho:

In [4]:
train.head()

,review,positive
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [5]:
train.shape

(24984, 2)

Com isso, percebemos apenas duas informações no dataframe: a review propriamente dita, e se ela é positiva (1) ou negativa (0).

Realizando uma análise quantitativa dos dados, percebemos, pela média, que o dataset é bem equilibrado com relação ao número de análises positivas e negativas. Quanto mais próxima de 0.5 a média está, mais balanceados são os dados:

In [6]:
train.describe()

,positive
count,24984.00000
mean,0.49988
std,0.50001
min,0.00000
25%,0.00000
50%,0.00000
75%,1.00000
max,1.00000


Podemos garantir isso observando a distribuição da variável "positive":

In [7]:
train["positive"].value_counts()

0    12495
1    12489
Name: positive, dtype: int64

In [8]:
train.isnull().sum()

review      0
positive    0
dtype: int64

Note, também, que não há dados faltantes na base de dados, o que deve proporcionar uma análise mais precisa.

## 2.3. Tratamento inicial dos dados:

Inicialmente, vamos procurar por reviews duplicadas no dataframe:

In [9]:
train[train.duplicated(["review"], keep = False)].sort_values(by = "review")

,review,positive
24826,'Dead Letter Office' is a low-budget film abou...,0
18434,'Dead Letter Office' is a low-budget film abou...,0
8122,".......Playing Kaddiddlehopper, Col San Fernan...",1
11731,".......Playing Kaddiddlehopper, Col San Fernan...",1
21968,"<br /><br />Back in his youth, the old man had...",0
...,...,...
21448,"in this movie, joe pesci slams dunks a basketb...",0
10996,it's amazing that so many people that i know h...,1
10993,it's amazing that so many people that i know h...,1
19524,this movie begins with an ordinary funeral... ...,0


Como a chance de essas duplicatas ocorrerem de verdade é extremamente baixa (dois textos idênticos serem escritos por pessoas distintas), podemos considerá-las como erros na coleta de dados. Por isso, serão removidas:

In [10]:
train = train.drop_duplicates("review", keep = "first")
train.shape

(24888, 2)

Como já observado, o dataset possui duas colunas: uma com as reviews, e uma com a variável target "positive". Assim, vamos criar duas listas, separando-as:

In [11]:
Y_train = np.array(train.positive.tolist())
X_train = train.review.tolist()

## 2.4. Tratamento e padronização dos textos:

Vamos observar alguns textos aleatórios, para identificarmos elementos que devem ser removidos, de modo a padronizar os dados e retirar itens desnecessários:

In [12]:
numbers = np.random.randint(24887, size=(5))
for i in numbers:
    print(X_train[i], "\n")

Sitting, TypingÂ… Nothing is the latest "what if?" fest offered by Vincenzio Natali, and starring David Hewlitt and Andrew Miller as two losers. One is having relationship problems, got canned from his job (because of relationship problems) and the police are out to get him (because of his job and his relationship problems). The other guy is a agoraphobic who refuses to go outside his home, is met by a bothersome girl guide who calls on her Mom to claim she was molested when he doesn't buy cookies from him. Oh yeah, the police are after him too, after the Mom of the girl scout call them in to arrest him.<br /><br />Man, what a day.<br /><br />What if you could make all of this disappear? That is the whole premise behind 'Nothing'. The two fools realize, the cops, the girl scout, the cars, the lawn, the road, everythingÂ… disappear. There's nothing but white space! This is an interesting concept I thought. I also looked at the time of this, 30 minutes had gone in the movie, and I still 

Note que há diversos elementos que devem ser removidos, o principal sendo "<br />"; e outros a serem corrigidos. Como dito no início do notebook, a importação do pacote $\textit{ftfy}$ tem objetivo de realizar essas correções.

É recomendado que a função de limpeza utilizada no uso do modelo de representação, ou seja, no Doc2Vec, seja idêntica à utilizada no treinamento do modelo de representação. Assim, a função a seguir será copiada do notebook $\textit{"Introdução ao Doc2Vec"}$ fornecido:

In [13]:
def clean(text):
    txt=text.replace("<br />"," ") #retirando tags
    txt=fix_text(txt) #consertando Mojibakes (Ver https://pypi.org/project/ftfy/)
    txt=txt.lower() #passando tudo para minúsculo
    txt=txt.translate(str.maketrans('', '', string.punctuation)) #retirando toda pontuação
    txt=txt.replace(" — ", " ") #retirando hífens
    txt=re.sub("\d+", ' <number> ', txt) #colocando um token especial para os números
    txt=re.sub(' +', ' ', txt) #deletando espaços extras
    return txt

Aplicando a função em todas as avaliações:

In [14]:
%%time
X_train = [clean(x) for x in X_train]

CPU times: user 1min 8s, sys: 3.6 ms, total: 1min 8s
Wall time: 1min 8s


Exibindo os mesmos textos após serem padronizados:

In [15]:
for i in numbers:
    print(X_train[i], "\n")

sitting typing… nothing is the latest what if fest offered by vincenzio natali and starring david hewlitt and andrew miller as two losers one is having relationship problems got canned from his job because of relationship problems and the police are out to get him because of his job and his relationship problems the other guy is a agoraphobic who refuses to go outside his home is met by a bothersome girl guide who calls on her mom to claim she was molested when he doesnt buy cookies from him oh yeah the police are after him too after the mom of the girl scout call them in to arrest him man what a day what if you could make all of this disappear that is the whole premise behind nothing the two fools realize the cops the girl scout the cars the lawn the road everything… disappear theres nothing but white space this is an interesting concept i thought i also looked at the time of this <number> minutes had gone in the movie and i still had an hour left in the movie could the <number> actor

Transformando os textos em vetores, cada posição com uma palavra:

In [16]:
X_train = [x.split() for x in X_train]

Com isso, os textos estarão uniformizados e prontos para serem utilizados no Doc2Vec.

# 3. Aplicando o modelo Doc2Vec:

Para esse exercício, já temos um Doc2Vec pré-treinado. Vamos importá-lo:

In [17]:
d2v = Doc2Vec.load("../input/sentiment-analysis-pmr3508/doc2vec")

Agora, devemos atualizar os pesos da rede neural do Doc2Vec para os textos desse dataset. A seguir, vamos fixar uma seed para possibilitar resultados consistentes, e iremos definir que a descida do gradiente dê 30 passos.

In [18]:
def emb(text, model, normalize = False): 
    model.random.seed(42)
    x = model.infer_vector(text, steps = 20)
    
    if normalize: 
        return(x/np.sqrt(x@x))
    
    else: 
        return(x)

Utilizando a função acima, vamos obter os vetores correspondentes aos textos:

In [19]:
%%time
X_train = [emb(x, d2v) for x in X_train] 
X_train = np.array(X_train)

CPU times: user 4min 5s, sys: 442 ms, total: 4min 5s
Wall time: 4min 5s


# 4. Criação e Treino das Redes Neurais:

## 4.1. Utilizando o SKLearn

### 4.1.1. Rede Neural 1 (SKLearn, 1 Camada)

Primeiro vamos inicializar a rede neural do pacote SKLearn, já adicionando a opção "early_stopping", que interrompe o treinamento caso a medida de acurácia não melhore em um determinado número de iterações.

In [20]:
NN_SKLearn = neural_network.MLPClassifier(early_stopping = True)
NN_SKLearn.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': True,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

Vamos usar GridSearchCV para escolher as combinações de hiperparâmetros ideais. O "alpha" poderá assumir dois valores mais comuns, "learning_rate" será testado com as duas métricas possíveis, e o tamanho das camadas deve variar de 1 a 100, de modo a encontrar o melhor nesse intervalo. A métrica utilizada será a AUC, a área embaixo do gráfico da curva ROC.

In [21]:
grid_params_NN_SKLearn = {"alpha":[0.0001, 0.001], "learning_rate":["constant", "adaptive"], 
                          "hidden_layer_sizes":[i for i in range(1, 101)]}
grid_NN_SKLearn = GridSearchCV(NN_SKLearn, grid_params_NN_SKLearn, n_jobs = -1, cv = 2, verbose = 1, scoring = "roc_auc")

In [22]:
%%time
grid_NN_SKLearn.fit(X_train, Y_train)
print("Hiperparâmetros:", grid_NN_SKLearn.best_estimator_)
print("Melhor score:", grid_NN_SKLearn.best_score_)

Fitting 2 folds for each of 400 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  7.8min finished


Hiperparâmetros: MLPClassifier(early_stopping=True, hidden_layer_sizes=99,
              learning_rate='adaptive')
Melhor score: 0.8893809288397777
CPU times: user 7.83 s, sys: 439 ms, total: 8.27 s
Wall time: 7min 51s


In [23]:
NN_SKLearn = grid_NN_SKLearn.best_estimator_

### 4.1.2. Rede Neural 2 (SKLearn, 2 Camadas)

Vamos usar um tratamento semelhante ao realizado acima. No entanto, vamos restringir nossa busca a um intervalo de 50 a 100 neurônios, com um step de 10, na primeira camada, e de 20 até o número de neurônios da primeira.

In [24]:
NN_SKLearn_2 = neural_network.MLPClassifier(early_stopping = True)

In [25]:
grid_params_NN_SKLearn_2 = {"alpha":[0.0001, 0.001], "learning_rate":["constant", "adaptive"], 
                          "hidden_layer_sizes":[(i, j) for i in range(40, 101, 10) for j in range(20, i, 10)]}
grid_NN_SKLearn_2 = GridSearchCV(NN_SKLearn_2, grid_params_NN_SKLearn_2, n_jobs = -1, cv = 2, verbose = 1, scoring = "roc_auc")

In [26]:
%%time
grid_NN_SKLearn_2.fit(X_train, Y_train)
print("Hiperparâmetros:", grid_NN_SKLearn_2.best_estimator_)
print("Melhor score:", grid_NN_SKLearn_2.best_score_)

Fitting 2 folds for each of 140 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   31.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:  3.7min finished


Hiperparâmetros: MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 80))
Melhor score: 0.8904698207347547
CPU times: user 12.7 s, sys: 222 ms, total: 12.9 s
Wall time: 3min 48s


In [27]:
NN_SKLearn_2 = grid_NN_SKLearn_2.best_estimator_

## 4.2. Utilizando Tensorflow e Keras

Observando tanto o notebook fornecido por Felipe Maia quanto algumas sugestões no website do Tensorflow, é recomendado que a primeira rede neural criada com essa biblioteca seja do modelo sequencial. Nele, vamos criando e empilhando cada camada da rede. Precisamos, então, definir dois parâmetros: o número de camadas, e o número de hidden units em cada uma.

Posteriormente, vamos criar outro modelo, utilizando duas camadas ocultas, com um algoritmo de Random Search, de modo a otimizar os hiperparâmetros pertinentes. Estes são dois valores de regularização (l1, l2), e o número de neurônios em cada camada. Vamos fixar o número de iterações do "early stopping" em dez, assim como ocorre no SKLearn.

### 4.2.1. Rede Neural 3 (Modelo Sequencial 1)

Aqui, seguimos testes de tentativa e erro, de acordo com o que se pode imaginar que trará bons resultados. Utilizaremos o mesmo número de camadas e de neurônios por camada que atingiram o melhor resultado com o SKLearn, para visualizar a diferença em performance entre os dois métodos.

In [28]:
n_features = X_train.shape[1] #Número de atributos que serão utilizados

ops.reset_default_graph() #Resetando as redes neurais treinadas até aqui (importante para testes)

NN_S1 = Sequential()
NN_S1.add(Dense(100, activation = "relu", input_shape = (n_features,)))
NN_S1.add(Dense(90, activation = "relu"))
NN_S1.add(Dense(1, activation = "sigmoid"))

Podemos observar o número de parâmetros de cada rede definida:

In [29]:
NN_S1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               5100      
_________________________________________________________________
dense_1 (Dense)              (None, 90)                9090      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 91        
Total params: 14,281
Trainable params: 14,281
Non-trainable params: 0
_________________________________________________________________


Agora, os modelos precisam de duas funções para serem compilados: uma função loss e um otimizador. Como loss, vamos utilizar "binary_crossentropy", já que é um modelo de classificação binário, e a saída do modelo é uma probabilidade. Além disso, assim como na Rede Neural do SKLearn, vamos utilizar a AUC como métrica:

In [30]:
NN_S1.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = [keras.metrics.AUC()])

Para treinar os modelos, vamos definir o tamanho dos "lotes", além da fração dos dados que serão separados para validação e o número máximo de iterações (epochs). Para separar os dados de validação, usamos a seguinte função:

In [31]:
X_train1, X_val, Y_train1, Y_val = train_test_split(X_train, Y_train, test_size=0.20)

In [32]:
history_S1 = NN_S1.fit(X_train1, Y_train1, validation_data = (X_val, Y_val), epochs = 50, batch_size = 96, 
                          shuffle=True, verbose=1)

Epoch 1/50
208/208 [==============================] - 1s 3ms/step - loss: 0.4647 - auc: 0.8648 - val_loss: 0.4265 - val_auc: 0.8889
Epoch 2/50
208/208 [==============================] - 0s 2ms/step - loss: 0.4151 - auc: 0.8925 - val_loss: 0.4178 - val_auc: 0.8910
Epoch 3/50
208/208 [==============================] - 0s 2ms/step - loss: 0.4030 - auc: 0.8989 - val_loss: 0.4158 - val_auc: 0.8918
Epoch 4/50
208/208 [==============================] - 0s 2ms/step - loss: 0.3943 - auc: 0.9033 - val_loss: 0.4137 - val_auc: 0.8927
Epoch 5/50
208/208 [==============================] - 0s 2ms/step - loss: 0.3845 - auc: 0.9082 - val_loss: 0.4184 - val_auc: 0.8926
Epoch 6/50
208/208 [==============================] - 0s 2ms/step - loss: 0.3749 - auc: 0.9129 - val_loss: 0.4173 - val_auc: 0.8923
Epoch 7/50
208/208 [==============================] - 0s 2ms/step - loss: 0.3665 - auc: 0.9171 - val_loss: 0.4268 - val_auc: 0.8900
Epoch 8/50
208/208 [==============================] - 0s 2ms/step - loss: 0.

Note que adotar o mesmo número de camadas que deu o melhor resultado com o SKLearn não necessariamente produzirá o melhor resultado com Tensorflow/Keras. A seguir, no próximo modelo, vamos efetivamente otimizar os hiperparâmetros para esses módulos.

### 4.2.2. Rede Neural 4 (Otimizada, Random Search)

Para regularização, utilizaremos uma estratégia do Tensorflow chamada "Early Stopping", ou seja, se a métrica de acurácia não melhorar dentro de um determinado número de iterações, o programa interrompe o treinamento, mesmo que o número total de epochs não tenha sido atingindo. Além disso, vamos restringir os pesos da rede, fazendo uso de duas penalizações, que devemos encontrar o valor ideal. Finalmente, temos que encontrar o tamanho ideal das duas camadas ocultas. No total, portanto, temos quatro números a serem otimizados.

Faremos uso de uma metodologia de Random Search. A partir de uma variedade de valores, será escolhida aleatoriamente uma combinação de quatro valores: um para o número de perceptrons de cada camada escondida (n1 e n2); e um para cada penalização (l1 e l2). No total, serão escolhidas 200 dessas combinações.

In [33]:
n_iter = 200

n_neurons = []
pen = []

for i in range(n_iter): #Gerando as 200 combinações:
    n1 = random.randrange(25, 101, 5) #Assim como feito na Rede Neural 2, vamos pegar apenas uma amostragem dos valores entre 25 e 100.
    n2 = random.randrange(20, n1, 5)
    n_neurons.append((n1, n2))
    
    l1 = random.choice([0, 1e-15, 1e-10, 1e-5, 1e-3, 1e-2, 1e-1])
    l2 = random.choice([0, 1e-15, 1e-10, 1e-5, 1e-3, 1e-2, 1e-1])
    pen.append((l1, l2))

Agora, vamos criar um DataFrame com as combinações obtidas:

In [34]:
params = {"n_neurons": n_neurons, "penalizações": pen, "iterações": n_iter*[None], "auc": n_iter*[None]}
params = pd.DataFrame(params)
params = params[["n_neurons", "penalizações", "iterações", "auc"]]
params.head()

,n_neurons,penalizações,iterações,auc
0,"(70, 25)","(0, 0.001)",None,None
1,"(50, 25)","(1e-10, 0.01)",None,None
2,"(90, 45)","(0.001, 1e-10)",None,None
3,"(35, 30)","(1e-05, 0.1)",None,None
4,"(100, 80)","(1e-10, 0.1)",None,None


Note que os outputs na camada "n_neurons" são tuplas que representam o número de neurônios na primeira e na segunda camada escondida, respectivamente; as saídas em "penalizações" são tuplas com os valores das duas regularizações apresentadas. "iterações" e "auc" serão completadas em seguida. A primeira irá armazenar o número de iterações completas até a parada, devido ao Early Stopping. Já a segunda deve armazenar o score de AUC no Early Stopping.

Para completar a tabela, vamos definir uma função que cria modelos para treinamento:

In [35]:
def create_model(n_neurons = (10,10), pen = (.001, .001)):
    
    ops.reset_default_graph() #Resetando as redes neurais treinadas até aqui (importante para testes)
    
    model = Sequential()
    model.add(Dense(n_neurons[0], input_shape = (n_features,), activation = "relu", 
                    kernel_regularizer = regularizers.l1_l2(l1 = pen[0], l2 = pen[1]), 
                    bias_regularizer = regularizers.l1_l2(l1 = pen[0], l2 = pen[1])))
    model.add(Dense(n_neurons[1], activation = "relu", 
                    kernel_regularizer = regularizers.l1_l2(l1 = pen[0], l2 = pen[1]), 
                    bias_regularizer = regularizers.l1_l2(l1 = pen[0], l2 = pen[1])))
    model.add(Dense(1, activation = "sigmoid", 
                    kernel_regularizer = regularizers.l1_l2(l1 = pen[0], l2 = pen[1]), 
                    bias_regularizer = regularizers.l1_l2(l1 = pen[0], l2 = pen[1])))
    
    model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = [keras.metrics.AUC()])
    return model

Note que a diferença entre a definição dessa rede e da Rede Neural 3 é que, além de ter uma camada a mais, agora temos dois parâmetros que não estavam presentes: "kernel_regularizer" e "bias_regularizer", que são os valores de penalização que definimos. A explicação para os modelos de loss e metrics são os mesmos explicitados na Rede Neural 3.

Primeiramente, vamos definir o Early Stopping que será utilizado: se a AUC em relação ao conjunto de validação não melhorar em dez iterações, o programa para:

In [36]:
es = EarlyStopping(monitor = "val_auc", patience = 10)

Agora vamos criar um modelo para cada combinação adquirida com o Random Search, finalizando o preenchimento da tabela:

In [37]:
%%time
for i in tqdm(range(n_iter)):
    model = create_model(params.loc[i, "n_neurons"], params.loc[i, "penalizações"])
    
    history = model.fit(X_train1, Y_train1, epochs = 50, validation_data = (X_val, Y_val), 
                        batch_size = 96, shuffle = True, verbose = False, callbacks = [es]) 
    
    params.loc[i, "iterações"] = len(history.history["val_auc"])
    params.loc[i, "auc"] = history.history["val_auc"][-1]

100%|██████████| 200/200 [20:26<00:00,  6.13s/it]

CPU times: user 36min 47s, sys: 4min 36s, total: 41min 23s
Wall time: 20min 26s


Vamos reordenar o dataframe, de modo que os modelos apareçam em ordem decrescente de acurácia:

In [38]:
params = params.sort_values("auc", ascending = False)

Visualizando os dez melhores resultados:

In [39]:
params.head(10)

,n_neurons,penalizações,iterações,auc
15,"(60, 40)","(1e-15, 0.001)",11,0.894834
75,"(50, 35)","(1e-10, 0.001)",11,0.894826
47,"(50, 35)","(0, 0.001)",11,0.894576
104,"(55, 45)","(0, 0.001)",11,0.894392
111,"(75, 45)","(1e-10, 0.001)",11,0.8943
191,"(70, 60)","(1e-05, 0.001)",11,0.894221
126,"(35, 30)","(1e-05, 0)",11,0.894178
113,"(65, 60)","(1e-05, 0.001)",11,0.894156
67,"(30, 25)","(0, 1e-15)",11,0.894058
183,"(60, 25)","(1e-15, 1e-05)",11,0.893933


Com a melhor rede encontrada, definimos e treinamos nosso modelo final:

In [40]:
n_neurons = params["n_neurons"][0]
pen = params["penalizações"][0]
epochs = params["iterações"][0]

NN_TF = create_model(n_neurons, pen)

history = NN_TF.fit(X_train1, Y_train1, epochs = epochs, validation_data = (X_val, Y_val), 
                        batch_size = 96, shuffle = True, verbose = 1)

Epoch 1/11
208/208 [==============================] - 1s 3ms/step - loss: 0.5638 - auc: 0.8568 - val_loss: 0.5020 - val_auc: 0.8873
Epoch 2/11
208/208 [==============================] - 0s 2ms/step - loss: 0.4858 - auc: 0.8911 - val_loss: 0.4795 - val_auc: 0.8904
Epoch 3/11
208/208 [==============================] - 0s 2ms/step - loss: 0.4660 - auc: 0.8954 - val_loss: 0.4690 - val_auc: 0.8909
Epoch 4/11
208/208 [==============================] - 0s 2ms/step - loss: 0.4544 - auc: 0.8974 - val_loss: 0.4601 - val_auc: 0.8928
Epoch 5/11
208/208 [==============================] - 0s 2ms/step - loss: 0.4443 - auc: 0.9000 - val_loss: 0.4561 - val_auc: 0.8936
Epoch 6/11
208/208 [==============================] - 0s 2ms/step - loss: 0.4375 - auc: 0.9018 - val_loss: 0.4491 - val_auc: 0.8945
Epoch 7/11
208/208 [==============================] - 0s 2ms/step - loss: 0.4319 - auc: 0.9036 - val_loss: 0.4496 - val_auc: 0.8934
Epoch 8/11
208/208 [==============================] - 0s 2ms/step - loss: 0.

# 5. Modelos alternativos

## 5.1. KNN

Vamos definir o KNN e encontrar os melhores hiperparâmetros utilizando o método GridSearchCV. Buscaremos o "k" ótimo no intervalo de 3 a 35.

In [41]:
knn = KNeighborsClassifier(algorithm = "auto", leaf_size = 30, n_jobs = -1)

In [42]:
grid_params_knn = {"p":[1, 2], "n_neighbors":[i for i in range(3, 36)]}
grid_knn = GridSearchCV(knn, grid_params_knn, cv = 2, n_jobs = -1, verbose = 1, scoring = "roc_auc")

In [43]:
%%time
grid_knn.fit(X_train, Y_train)

Fitting 2 folds for each of 66 candidates, totalling 132 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 132 out of 132 | elapsed: 23.1min finished


CPU times: user 907 ms, sys: 192 ms, total: 1.1 s
Wall time: 23min 8s


GridSearchCV(cv=2, estimator=KNeighborsClassifier(n_jobs=-1), n_jobs=-1,
             param_grid={'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                         14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                                         24, 25, 26, 27, 28, 29, 30, 31, 32, ...],
                         'p': [1, 2]},
             scoring='roc_auc', verbose=1)

In [44]:
print("Hiperparâmetros:", grid_knn.best_estimator_)
print("Melhor score:", grid_knn.best_score_)

Hiperparâmetros: KNeighborsClassifier(n_jobs=-1, n_neighbors=35)
Melhor score: 0.8503667543153675


Mantendo o melhor modelo:

In [45]:
knn = grid_knn.best_estimator_

## 5.2. SVM

Definindo o Support Vector Machine:

In [46]:
svm = svm.SVC(probability = True)

Também utilizando o GridSearchCV para encontrar hiperparâmetros ideais:

In [47]:
grid_params_svm = {"C":[0.1, 1, 10],  
              "kernel":["linear", "rbf"], "probability":[True]}  
  
grid_svm = GridSearchCV(svm, grid_params_svm, cv = 2, verbose = 1, n_jobs = -1, scoring = "roc_auc") 

In [48]:
%%time
grid_svm.fit(X_train, Y_train) 

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  9.3min finished


CPU times: user 5min 21s, sys: 258 ms, total: 5min 21s
Wall time: 14min 36s


GridSearchCV(cv=2, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'],
                         'probability': [True]},
             scoring='roc_auc', verbose=1)

In [49]:
print("Hiperparâmetros:", grid_svm.best_estimator_)
print("Melhor score:", grid_svm.best_score_)

Hiperparâmetros: SVC(C=1, probability=True)
Melhor score: 0.8871900877790198


Mantendo o melhor modelo encontrado:

In [50]:
svm = grid_svm.best_estimator_

## 5.3. Regressão Logística

Vamos criar um modelo de regressão logística, selecionando os melhores hiperparâmetros também com GridSearchCV:

In [51]:
%%time
logreg = LogisticRegression(solver='liblinear',random_state=36)
grid_params_logreg = {"C":[x for x in np.linspace(0,10,100)], 
                     "penalty":['l2', 'l1']}

grid_logreg = GridSearchCV(logreg, grid_params_logreg, scoring = "roc_auc", 
                            cv = 2, n_jobs = -1, verbose = 2)

grid_logreg.fit(X_train, Y_train) 

Fitting 2 folds for each of 200 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  1.2min finished


CPU times: user 2.34 s, sys: 0 ns, total: 2.34 s
Wall time: 1min 12s


GridSearchCV(cv=2,
             estimator=LogisticRegression(random_state=36, solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': [0.0, 0.10101010101010101, 0.20202020202020202,
                               0.30303030303030304, 0.40404040404040403,
                               0.5050505050505051, 0.6060606060606061,
                               0.7070707070707071, 0.8080808080808081,
                               0.9090909090909091, 1.0101010101010102,
                               1.1111111111111112, 1.2121212121212122,
                               1.31...1414141414141,
                               1.5151515151515151, 1.6161616161616161,
                               1.7171717171717171, 1.8181818181818181,
                               1.9191919191919191, 2.0202020202020203,
                               2.121212121212121, 2.2222222222222223,
                               2.323232323232323, 2.4242424242424243,
                               2.525

In [52]:
print("Hiperparâmetros:", grid_svm.best_estimator_)
print("Melhor score:", grid_svm.best_score_)

Hiperparâmetros: SVC(C=1, probability=True)
Melhor score: 0.8871900877790198


Mantendo o melhor modelo encontrado:

In [53]:
logreg = grid_logreg.best_estimator_

# 6. Tratamento dos dados de validação e de teste

Vamos tratar os dados de validação e de teste da mesma maneira que fizemos com os dados de treino.

Importando validação e teste, respectivamente:

In [54]:
data_test1 = pd.read_csv("../input/sentiment-analysis-pmr3508/data_test1.csv")
data_test2_X = pd.read_csv("../input/sentiment-analysis-pmr3508/data_test2_X.csv")

Vamos observar o início de cada um deles:

In [55]:
data_test1.head()

,review,positive
0,This was one of the most emotional movies I ha...,1
1,New Year's Day. The day after consuming a few ...,0
2,Before launching into whether this film is wor...,0
3,Critters 4 is a good movie. A bit of a twist t...,1
4,"For connoisseurs of bad movies, Galaxina is a ...",0


In [56]:
data_test2_X.head()

,review
0,"How is it in this day and era, people are stil..."
1,"I mean let's face it, all you have to do in mo..."
2,"""RVAM""'s reputation preceded it. I first heard..."
3,A lot of the negative reviews here concentrate...
4,A SOUND OF THUNDER. One of the greatest short ...


Note que data_test1 realmente é o conjunto de validação, por possuir tanto as reviews quanto a classificação. Já data_test2_X só possui as reviews. Essa base será utilizada para gerar um arquivo de predição para submissão à competição.

Podemos observar que os dados de validação são bem balanceados, assim como os dados de treino. Há um número similar de avaliações positivas e negativas:

In [57]:
data_test1["positive"].value_counts()

0    6308
1    6184
Name: positive, dtype: int64

Vamos observar se há algum dado faltante em um dos dois datasets:

In [58]:
data_test1.isnull().sum()

review      0
positive    0
dtype: int64

In [59]:
data_test2_X.isnull().sum()

review    0
dtype: int64

Como não há dados faltantes, continuamos nossa análise.

Agora, buscamos por reviews duplicadas na base de validação (não fazemos isso na base de teste pois ela deve permanecer no tamanho original):

In [60]:
data_test1[data_test1.duplicated(["review"], keep = False)].sort_values(by = "review")

,review,positive
262,(Spoilers)<br /><br />Oh sure it's based on Mo...,0
6365,(Spoilers)<br /><br />Oh sure it's based on Mo...,0
6747,A lot about USA The Movie can be summed up in ...,1
5834,A lot about USA The Movie can be summed up in ...,1
4408,After reading previews for this movie I though...,1
...,...,...
11921,"What a shocker. For starters, I couldn't stand...",0
11720,What the heck is this about? Kelly (jennifer) ...,0
10042,What the heck is this about? Kelly (jennifer) ...,0
6890,this movie sucks. did anyone notice that the e...,0


Há algumas duplicatas, mas, assim como ocorreu com a base de treino, a chance de terem sido realmente submetidas mais de uma vez é muito baixa. Provavelmente se deram por um erro na coleta dos dados. Por isso, devemos manter apenas uma cópia de cada um.

In [61]:
data_test1 = data_test1.drop_duplicates("review", keep = "first")
data_test1.shape

(12441, 2)

Separando as duas variáveis do conjunto de validação:

In [62]:
Y_test = np.array(data_test1.positive.tolist())
X_test = data_test1.review.tolist()

Agora vamos limpar os textos de ambas as bases, utilizando a função "clean", definida no início do notebook:

In [63]:
%%time
X_test = [clean(x) for x in X_test]
X_test2 = data_test2_X.review.tolist()
X_test2 = np.array([clean(x) for x in X_test2])

CPU times: user 1min 7s, sys: 0 ns, total: 1min 7s
Wall time: 1min 7s


Tokenizando os textos:

In [64]:
X_test = [x.split() for x in X_test]
X_test2 = [x.split() for x in X_test2]

Aplicando o modelo Doc2Vec:

In [65]:
%%time
X_test = [emb(x, d2v) for x in X_test] 
X_test = np.array(X_test)
X_test2 = [emb(x, d2v) for x in X_test2] 
X_test2 = np.array(X_test2)

CPU times: user 4min 7s, sys: 0 ns, total: 4min 7s
Wall time: 4min 7s


# 7. Validação e Score dos métodos definidos:

Agora, vamos gerar predições, utilizando o conjunto de validação, a partir dos modelos definidos até aqui: as redes neurais, o classificador KNN, a Support Vector Machine e a Regressão Logística. Vamos compará-las com a classificação correta. Assim, descobrimos aquele com maior acurácia:

In [66]:
print("AUC --- NN_SKLearn: {:.4f}".format(roc_auc_score(Y_test, NN_SKLearn.predict_proba(X_test)[:,1])))
print("AUC --- NN_SKLearn_2: {:.4f}".format(roc_auc_score(Y_test, NN_SKLearn_2.predict_proba(X_test)[:,1])))
print("AUC --- NN_S1: {:.4f}".format(roc_auc_score(Y_test, NN_S1.predict(X_test).squeeze())))
print("AUC --- NN_TF: {:.4f}".format(roc_auc_score(Y_test, NN_TF.predict(X_test).squeeze())))
print("AUC --- KNN: {:.4f}".format(roc_auc_score(Y_test, knn.predict_proba(X_test)[:,1])))
print("AUC --- SVM: {:.4f}".format(roc_auc_score(Y_test, svm.predict_proba(X_test)[:,1])))
print('AUCs --- Log. Reg.: {:.4f}'.format(roc_auc_score(Y_test, logreg.predict_proba(X_test)[:,1])))

AUC --- NN_SKLearn: 0.8880
AUC --- NN_SKLearn_2: 0.8918
AUC --- NN_S1: 0.8439
AUC --- NN_TF: 0.8930
AUC --- KNN: 0.8572
AUC --- SVM: 0.8925
AUCs --- Log. Reg.: 0.8835


Assim, temos que a SVM e a rede neural do Tensorflow produziram as melhores AUC's. Como A NN_TF teve um score levemente maior, vamos selecioná-la:

In [67]:
YtestPred = {"positive":NN_TF.predict(X_test2).squeeze()}
submission = pd.DataFrame(YtestPred)
submission.to_csv("submission.csv", index = True, index_label = "Id")

# 8. Conclusões

Submetidos os resultados à competição, podemos perceber que diversos modelos diferentes, se devidamente treinados e otimizados, podem atingir resultados semelhantes. Dito isso, é provável que as redes neurais sejam mais maleáveis, além de, no geral, mais rápidas de serem treinadas. O classificador KNN pode obter resultados interessantes, embora não seja interessante em termos de poder computacional. As Support Vector Machines também podem demorar muito para serem treinadas, dependendo do número de parâmetros. Aqui, foram necessários 16 minutos para um 2-fold cross validation e algumas poucas combinações de parâmetros.

De modo geral, é sempre interessante aplicar modelos como o SVM, mas aqui, as redes neurais (especialmente do Tensorflow), foram os melhores modelos.